<a href="https://colab.research.google.com/github/mmsamiei/lets-pytorch/blob/master/simple_cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision.datasets

In [0]:
import torchvision.transforms as transforms
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1.0,))])
MNIST_dataset = torchvision.datasets.MNIST('./mnist', transform=trans)

In [0]:
from torch.utils.data import DataLoader
train_dl = DataLoader(MNIST_dataset, batch_size=128)

In [0]:
from torch import nn
class ImanNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, 3)
        self.conv2 = nn.Conv2d(3, 6, 3)
        self.pool1 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(6, 9, 5)
        self.lin = nn.Linear(576, 10)
        self.softmax = nn.Softmax(1)

    def forward(self, x):
        temp = self.conv1(x)
        temp = nn.functional.relu(temp)
        temp = self.conv2(temp)
        temp = nn.functional.relu(temp)
        temp = self.pool1(temp)
        temp = self.conv3(temp)
        temp = nn.functional.relu(temp)
        temp = temp.view(-1, 576)
        temp = self.lin(temp)
        out = self.softmax(temp)
        return out

In [0]:
test = torch.randn(1,1,28,28)

In [122]:
model = ImanNet()
model(test)

tensor([[0.1055, 0.0977, 0.1070, 0.0914, 0.0856, 0.1202, 0.0983, 0.0903, 0.0961,
         0.1079]], grad_fn=<SoftmaxBackward>)

In [0]:
opt = torch.optim.Adam(model.parameters())

In [124]:
loss_arr = []
for epoch in range(3):
  for xb,yb in train_dl:
    out = model(xb)
    loss = nn.functional.cross_entropy(out, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()
    loss_arr.append(loss.item())
  print("epoch ",epoch,", loss = ",loss)

epoch  0 , loss =  tensor(1.6238, grad_fn=<NllLossBackward>)
epoch  1 , loss =  tensor(1.6036, grad_fn=<NllLossBackward>)
epoch  2 , loss =  tensor(1.5770, grad_fn=<NllLossBackward>)


In [125]:
correct = 0
total = 0
with torch.no_grad():
    for data in train_dl:
        images, labels = data
        out = model(images)
        _, predicted = torch.max(out.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 60000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 60000 test images: 86 %
